In [ ]:
# Conectamos con el google drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Instalamos las dependencias de nuestro entorno de trabajo 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://apache.osuosl.org/spark/spark-3.1.3/spark-3.1.3-bin-hadoop2.7.tgz
!tar xf spark-3.1.3-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install koalas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 390 kB 28.8 MB/s 


In [ ]:
!ls -a

.   .config  sample_data		spark-3.1.3-bin-hadoop2.7.tgz
..  drive    spark-3.1.3-bin-hadoop2.7


In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.3-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
%matplotlib inline
import databricks.koalas as ks
from pathlib import Path

In [ ]:
def importar(path:Path, format:str): 
  '''
  This function imports files with spark and transforms them into DataFrame using the koala library

  Arguments:
    :: path: 'path' path where the file is stored
    :: format: 'str' file format 

  Returns: 
  ---------
  Dataframe and print shape 
  '''

  df = spark.read.load(path, format=format)
  df = df.to_koalas()
  print(df.shape)

  return df


In [ ]:
# Import all the datasets and print their shape
review = importar("/content/drive/MyDrive/HENRY_TRABAJO_GRUPAL/Dataset_Yelp/review.json", "json")
user = importar("/content/drive/MyDrive/HENRY_TRABAJO_GRUPAL/Dataset_Yelp/user.json", "json")
business = importar("/content/drive/MyDrive/HENRY_TRABAJO_GRUPAL/Dataset_Yelp/business.json", "json")
chechin = importar("/content/drive/MyDrive/HENRY_TRABAJO_GRUPAL/Dataset_Yelp/checkin.json", "json")
tip= importar('/content/drive/MyDrive/HENRY_TRABAJO_GRUPAL/Dataset_Yelp/tip.json', "json")

(6990280, 9)
(1987897, 22)
(150346, 14)
(131930, 2)
(908915, 5)


In [ ]:
dataset_dict = {'review':review , 
                'user': user, 
                'business':business, 
                'chechin':chechin, 
                'tip': tip}

In [ ]:
tip.columns

Index(['business_id', 'compliment_count', 'date', 'text', 'user_id'], dtype='object')

In [ ]:
tip.info()

<class 'databricks.koalas.frame.DataFrame'>
Int64Index: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   business_id       908915 non-null  object
 1   compliment_count  908915 non-null  int64 
 2   date              908915 non-null  object
 3   text              908915 non-null  object
 4   user_id           908915 non-null  object
dtypes: int64(1), object(4)

In [ ]:
review.info()

<class 'databricks.koalas.frame.DataFrame'>
Int64Index: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   business_id  6990280 non-null  object 
 1   cool         6990280 non-null  int64  
 2   date         6990280 non-null  object 
 3   funny        6990280 non-null  int64  
 4   review_id    6990280 non-null  object 
 5   stars        6990280 non-null  float64
 6   text         6990280 non-null  object 
 7   useful       6990280 non-null  int64  
 8   user_id      6990280 non-null  object 
dtypes: float64(1), int64(3), object(5)

## EDA de Review

In [ ]:
def unique_values(data): 
  '''
  This function detects the unique values in the dataset

  Arguments:
    :: data: 'DataFrame' 

  Return: 
  ---------
  Number of unique values and display
  '''

  columns = data.columns 

  print('----Unique Values in DataFrame----','\n',data.nunique())
  unique_values_viz = data.nunique().plot(kind = 'bar', title='Number of Unique Values in DataFrame')

  for column in columns: 
    if data[column].nunique() < 20:
      unique_values = data[column].unique()
      print('\n''----Unique Values in column----','\n',unique_values)

  return unique_values_viz

In [ ]:
unique_values(tip)

----Unique Values in DataFrame---- 
 business_id         106193
compliment_count         7
date                906694
text                851033
user_id             301758
dtype: int64

----Unique Values in column---- 
 0    0
1    6
2    5
3    1
4    3
5    2
6    4
Name: compliment_count, dtype: int64


 - shape
 - info()
 - describe()
 - isnull().sum()
 - Duplicate.sum()
 - value_counts()

 - Outiliers --> Visualizar outliers

**Transformaciones**





### EDA tip

**Descripción de datos de columna:**

// string, texto del tip
"text": "Secret menu - fried chicken sando is da bombbbbbb Their zapatos are good too.",

// string, fecha cuando se escribio el tip YYYY-MM-DD
"date": "2013-09-20",

// entero, cuantos cumplidos totales tiene
"compliment_count": 172,

// string, 22 caracteres, id del negocio que se refiere al negocio en business.json
"business_id": "tnhfDv5Il8EaGSXZGiuQGg",

// string, 22 caracteres de id de usuario, que se refieren al usuario en user.json
"user_id": "49JhAJh8vSQ-vM4Aourl0g"

In [ ]:
tip.head()

,business_id,compliment_count,date,text,user_id
0,3uLgwr0qeCNMjKenHJwPGQ,0,2012-05-18 02:17:21,Avengers time with the ladies.,AGNUgVwnZUey3gcPCJ76iw
1,QoezRbYQncpRqyrLH6Iqjg,0,2013-02-05 18:35:10,They have lots of good deserts and tasty cuban...,NBN4MgHP9D3cw--SnauTkA
2,MYoRNLb5chwjQe3c_k37Gg,0,2013-08-18 00:56:08,It's open even when you think it isn't,-copOvldyKh1qr-vzkDEvw
3,hV-bABTK-glh5wj31ps_Jw,0,2017-06-27 23:05:38,Very decent fried chicken,FjMQVZjSqY8syIO-53KFKw
4,_uN0OudeJ3Zl_tf6nxg5ww,0,2012-10-06 19:43:09,Appetizers.. platter special for lunch,ld0AperBXk1h6UbqmM80zw


In [ ]:
tip.shape

(908915, 5)

In [ ]:
tip.info()

<class 'databricks.koalas.frame.DataFrame'>
Int64Index: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   business_id       908915 non-null  object
 1   compliment_count  908915 non-null  int64 
 2   date              908915 non-null  object
 3   text              908915 non-null  object
 4   user_id           908915 non-null  object
dtypes: int64(1), object(4)

In [ ]:
tip['length_text'] = tip['text'].apply(len)

/content/spark-3.1.3-bin-hadoop2.7/python/pyspark/sql/pandas/functions.py:392: UserWarning:

In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.



In [ ]:
tip['length_text'].max()

500

In [ ]:
tip['length_text'].hist(bins=10, title='Longitud de texto')

In [ ]:
tip.describe()

,compliment_count,length_text
count,908915.000000,908915.000000
mean,0.012525,62.580226
std,0.120763,57.828705
min,0.000000,1.000000
25%,0.000000,28.000000
50%,0.000000,49.000000
75%,0.000000,76.000000
max,6.000000,500.000000


In [ ]:
tip['date'] = ks.to_datetime(tip['date'])

/content/spark-3.1.3-bin-hadoop2.7/python/pyspark/sql/pandas/functions.py:392: UserWarning:

In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.



In [ ]:
tip.info()

<class 'databricks.koalas.frame.DataFrame'>
Int64Index: 908915 entries, 0 to 908914
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype     
---  ------            --------------   -----     
 0   business_id       908915 non-null  object    
 1   compliment_count  908915 non-null  int64     
 2   date              908915 non-null  datetime64
 3   text              908915 non-null  object    
 4   user_id           908915 non-null  object    
 5   length_text       908915 non-null  int64     
dtypes: datetime64(1), int64(2), object(3)

In [ ]:
tip.nunique().plot(kind = 'bar', title='Cantidad de Valores únicos')

In [ ]:
tip.unique().plot(kind = 'bar', title='Cantidad de Valores únicos')

AttributeError: ignored

In [ ]:
tip.isnull().sum()

In [ ]:
tip['compliment_count'].value_counts()

In [ ]:
tip['compliment_count'].value_counts().plot(kind = 'bar', title='Cantidad de valores únicos en compliment_count')

In [ ]:
min = tip['date'].min()
max = tip['date'].max()

print(f'Datos desde: {min}, hasta: {max}')

In [ ]:
tip['dates'] = ks.to_datetime(tip['date']).dt.date

In [ ]:
tip['time'] = ks.to_datetime(tip['date']).dt.strftime('%H:%M:%S')

In [ ]:
def is_empty(value):
  stripped = value.strip()
  if stripped == '':
    return 1
  else:
    return 0

In [ ]:
tip['empty_text'] = tip.text.apply(is_empty)

In [ ]:
tip.empty_text.value_counts()

## EDA de Review

In [ ]:
user.head()

In [ ]:
business.show()

In [ ]:
null_counts = review.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c)for c in review.columns]).toPandas().to_dict(orient='records')
print(f"We have {sum(null_counts[0].values()):d} null values in this dataset.")

In [ ]:
review.isna().sum()

In [ ]:
bussiness.head()

In [ ]:
chechin.info()

In [ ]:
chechin.isna().sum()

In [ ]:
chechin.head()

In [ ]:
review.isna().sum()

In [ ]:
review.info()

In [ ]:
tip.isna().sum()

In [ ]:
tip.head()

In [ ]:
user.isna().sum()

In [ ]:
package com.dkl.leanring.spark.sql
 
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SaveMode
import java.util.Properties
 
# /**
#    * Lea los datos de USER_T.csv e insértelos en la tabla mysql
#  */
object MysqlInsertDemo {
  def main(args: Array[String]): Unit = {
    val spark = SparkSession.builder().appName("MysqlInsertDemo").master("local").getOrCreate()
    val df = spark.read.option("header", "true").csv("src/main/resources/scala/USER_T.csv")
    df.show()
    val url = "jdbc:mysql://192.168.44.128:3306/hive?useUnicode=true&characterEncoding=utf-8"
    val prop = new Properties()
    prop.put("user", "root")
    prop.put("password", "Root-123456")
    df.write.mode(SaveMode.Append).jdbc(url, "USER_T", prop)
  }
}

In [ ]:
dicc = {
"Monday":"10:00-21:00",
"Tuesday":"10:00-21:00"
}





In [ ]:
Hours = []
for key in dicc.keys():
  if key == 'Monday':
    Hours.append([1,dicc[key].split('-')])
  if key == 'Tuesday':
    Hours.append([2,dicc[key].split('-')])
    dicc[key]
print(Hours)